In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

Load data and Show information

In [ ]:
DATA_PATH = "/content/drive/MyDrive/Undergrad/Semester-7/DWDM_Preprocessing-Assignment/Data"
df = pd.read_csv(os.path.join(DATA_PATH, "taxi_trip_data.csv"))
print(df.info())

In [ ]:
# Show first 5 rows
print(df.head())

# Introduction

The goal of this notebook is to clean and transform the data available for the purpose of later utilizing it in ML algorithms, or for data warehousing purposed.

The data cleaning process can begin to clear out outliers, missing values and other noise which might affect the results of the algorithm. 



## Background

Rides following similar paths in the past will likely take similar routes, and rides during the same hours of the day will also likely take roughly the same amount of time. This gives us a sort of rolling average for distance and time to make the calculation easier, what it doesn't give us, is how much of that distance is sitting in traffic, below 12mph, or driving at normal speeds above 12mph, nor does it account for sitting at red lights.

These values are hard to account for. While patterns can be detected when analysing the data through graphs and other visuals, it doesn't make for a very mathematical or repeatable prediction. We'll need the model to detect these patterns quickly and repeatably to get the most accurate predictions possible, which means some data, such as start and end times should be broken down into chunks that are easier for a machine to read such as the number of minutes per trip, the month, day, day of the week, and year (separately) .



## Dataset: NYC Taxi Trip Data - Google Public Data

This data set is a subset of the Google BigQuery public datasets - Nyc yellow taxi cab trips data set containing a random 10,000,000 rows of data. This dataset was extracted and uploaded for the purpose of experimenting with and learning regression models for price prediction. There is also a lot of room for data cleaning, outliers in the data, and plenty of data to work with for more realistic model training, testing, and validation.

### Data Attributes

| column            | type     | nullable | description                                                                                                                                                                                                                                          |
| ----------------- | -------- | -------- | ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| vendor\_id        | text     | required | A code indicating the TPEP provider that provided the record. 1= Creative Mobile Technologies, LLC; 2= VeriFone Inc                                                                                                                                  |
| pickup\_datetime  | datetime | nullable | The date and time when the meter was engaged.                                                                                                                                                                                                        |
| dropoff\_datetime | datetime | nullable | The date and time when the meter was disengaged.                                                                                                                                                                                                     |
| passenger\_count  | integer  | nullable | The number of passengers in the vehicle. This is a driver-entered value                                                                                                                                                                              |
| trip\_distance    | numeric  | nullable | The elapsed trip distance in miles reported by the taximeter.                                                                                                                                                                                        |
| rate\_code        | string   | nullable | The final rate code in effect at the end of the trip. 1= Standard rate 2=JFK 3=Newark 4=Nassau or Westchester 5=Negotiated fare 6=Group ride                                                                                                         |
| storeandfwd\_flag | string   | nullable | This flag indicates whether the trip record was held in vehicle memory before sending to the vendor, aka “store and forward,” because the vehicle did not have a connection to the server. Y= store and forward trip N= not a store and forward trip |
| payment\_type     | string   | nullable | A numeric code signifying how the passenger paid for the trip. 1= Credit card 2= Cash 3= No charge 4= Dispute 5= Unknown 6= Voided trip                                                                                                              |
| fare\_amount      | numeric  | nullable | The time-and-distance fare calculated by the meter                                                                                                                                                                                                   |
| extra             | numeric  | nullable | Miscellaneous extras and surcharges. Currently, this only includes the \\$0.50 and \\$1 rush hour and overnight charges.                                                                                                                             |
| mta\_tax          | numeric  | nullable | \\$0.50 MTA tax that is automatically triggered based on the metered rate in use                                                                                                                                                                     |
| tip\_amount       | numeric  | nullable | Tip amount – This field is automatically populated for credit card tips. Cash tips are not included                                                                                                                                                  |
| tolls\_amount     | numeric  | nullable | Total amount of all tolls paid in the trip.                                                                                                                                                                                                          |
| imp\_surcharge    | numeric  | nullable | \\$0.30 improvement surcharge assessed trips at the flag drop. The improvement surcharge began being levied in 2015.                                                                                                                                 |
| total\_amount     | numeric  | nullable | The total amount charged to passengers. Does not include cash tips                                                                                                                                                                                   |
| pickuplocationid  | string   | nullable | TLC Taxi Zone in which the taximeter was engaged                                                                                                                                                                                                     |
| dropofflocationid | string   | nullable | TLC Taxi Zone in which the taximeter was disengaged                        

## Plan for Features  

Here are the features of the current dataset that will be kept, as well as a few that will need to be created based on other features:  
- pickup_timestamp  
- dropoff_timestamp  
- trip_distance  
- fare_amount  
- extra  
- mta_tax  
- imp_surcharge  
- total_amount  
- pickup_location_id  
- dropoff_location_id  

# Data Analysis

The **correlation matrix** calculates how the change in one value effects a change in the other value, and assigns a value between -1 and 1 to that correlation.  
Let's review what those correlation values mean before we move on:  


### Correlation Matrix

The correlation matrix for all pairs of attributes is represented in the heatmap below:

* **-1:** A very strong negative correlation, when value A moves in one direction, value B moves in the opposite direction.  
* **0:**  No correlation between values A and B, when one moves, the other is not effected.  
* **1:**  A very strong positive correlation, as you can guess, this is the opposite of the negative correlation above. When value A moves in one direction, value B follows in the same direction.  

This value isn't related to the rate of change, only the direction of change. Value A moves up, and value B either stays, moves up, or moves down. 

In [ ]:
# Generating the correlation matrix
corr = df.corr()

In [ ]:
# Drawing the heatmap
fig, ax = plt.subplots(figsize=(15,15))
ax = sns.heatmap(corr, cmap='YlGnBu', annot=True, linewidths=0.5);

There is already a list of known values that we should keeep. The only remaining values are:  
- **vendor_id** - Vendor of data provider. This definitely won't be used for anything for our model here  
- **rate_code** - The rate code at the end of the trip. Used likely to track certain charges. Has a correlation with tolls and tips but not much with anything else.  
- **sotre_and_fwd_flag** - This is simply a fag that indicates whether a value was stored in vehicle memory before being recorded due to a lack of internet connection. This is useless to us, however, it's currently stored as a string and converting it to a value that can appear in a correlation matrix later might serve useful. While not likely, it could be possible that values are different from those not stored in memory, such as having a higher amount of errors, or some upload process might be altering values in an unexpected way.

In the end, only one column is being dropped right off the start and that's **Vendor ID**. 

In [ ]:
df = df.drop('vendor_id', axis=1)
df.head()

# Data Cleaning  

The following data cleaning steps are assessed and applied:
1. **Remove duplicate rows** - Carefully, as we only want to remove duplicate trips, not duplicates within the values themselves. These values are not required to be unique.
2. Check for **missing values**
3. Check for **zeros and empty strings**. These values won't be "missing" but still aren't valid. Very few columns in this data have valid zeros  
4.**Validate formatting** of data, especially dates  
5. **Strip and normalize strings** - our data doesn't contain any strings, so we can skip this.  

## Remove Duplicates

In [ ]:
# Remove duplicates - 
# Rename the dataframe from df to td for temporary data, thus not altering the original dataframe until much later. 
td = df.drop_duplicates()
# less than 1% dropped
print(f"{df.shape[0] - td.shape[0]} duplicate rows dropped. Thats {(df.shape[0] - td.shape[0]) / df.shape[0] * 100}%")
print(f"{td.shape[0]} rows remain.")

## Remove Missing Values

In [ ]:
# Checking for missing values
for col in td.columns:
    missing = td[col].isna().sum()
    print(f"Missing values in {col}: {missing}")

## Remove Zeros and Empty Strings

In [ ]:
# Checking for zeros in numeric columns
def check_for_zeros(td):
    for col in td.columns:
        zeros = td[td[col] == 0].shape[0]
        print(f"Zeros in {col}:{zeros}")
        
check_for_zeros(td)

**Changes applied so far ...**

* *passenger_count*, *trip_distance*, *fare_amount* and *total_amount* --- all contain zeros.

* It doesn't appear to be a large amount of the overall data.

* Without distance, we can't determine fare amount, even with distance, it's impossible to know which miles were driven above the 12mph threshold, and which were below.

* There isn't much of a choice but to drop these. However, **total_amount** can be corrected by simply adding all of the charge column values together, so I'll keep and fix these rows.

**Dropping rows with 0 values in columns where 0 is not allowed**

In [ ]:
# Dropping rows with 0 values in columns where 0 is not allowed
td = td.drop(['passenger_count'], axis=1)
td = td[td['trip_distance'] > 0]
td = td[td['fare_amount'] > 0]

check_for_zeros(td)

After dropping rows with zero values in other columns, there remains no zeros in total_amount, so no corrections are necessary here

In [ ]:
# Checking how much of the original data ramains
remaining = td.shape[0] / df.shape[0] * 100
print(f"Remaining amount of original dataset: {remaining}%")

## Validating Data Formats
  
Ensure that the dates are all readable date formats and exist in the same format such as mm/dd/yyyy, for example. 

In [ ]:
# Converting to an actual Python/Pandas datetime object ensures that the data is a valid datetime. 
# Then, we  move on to exploring the datetimes available.
td['pickup_datetime'] = pd.to_datetime(td['pickup_datetime'])
td['dropoff_datetime'] = pd.to_datetime(td['dropoff_datetime'])

print('Done.')

**Inference:** All datetime stamps in the dataset are correctly formatted

The datetime columns are now split up into meaninful columns. The only dropoff information we really need to keep is the hour, and even then, only to calculate the length of the trip. 

In [ ]:
td['year'] = pd.to_datetime(td['pickup_datetime']).dt.year
td['month'] = pd.to_datetime(td['pickup_datetime']).dt.month
td['day'] = pd.to_datetime(td['pickup_datetime']).dt.day
td['day_of_week'] = pd.to_datetime(td['pickup_datetime']).dt.dayofweek
td['hour_of_day'] = pd.to_datetime(td['pickup_datetime']).dt.hour

print('Done.')

## Cleaning Date and Time Data

### Validate Timestamps

In [ ]:
# Converting the datetime columns to a numpy array for vectorization
pickup_array = td['pickup_datetime'].values
dropoff_array = td['dropoff_datetime'].values

### Validate Trip Durations

In [ ]:
# Getting the new timedelta, this takes less than a second to complete compared to 15+ minutes with apply()
trip_duration = np.subtract(dropoff_array, pickup_array)

# Adding the resulting array to the dataframe in the trip_duration column
td['trip_duration'] = pd.Series(trip_duration)

# Converting the timedelta to number of seconds
td['trip_duration'] = td['trip_duration'].dt.total_seconds()

# Preview the results
td.head()

Now, the datetime columns can be dropped entirely. 

In [ ]:
td.drop(['pickup_datetime', 'dropoff_datetime'], axis=1, inplace=True)

Displaying the dataset's current state ...

In [ ]:
td.head()

Now that the dates have been broken down properly, a higher level of data clean-up can be performed.

* Any trips with a duration of 0 need to be dropped. These trips won't be useful, and are certainly due to a data entry error.  

* Investigate what years are available in this dataset, how much of the dataset each year makes up, and begin investigating whether we should keep all years, or only specific years by visualizing trends in fare amounts when compared to trip duration and distance. 

In [ ]:
td = td[td['trip_duration'] > 0]

In [ ]:
list_of_years = td.year.unique()
print(list_of_years)

In [ ]:
for year in list_of_years:
    year_amount = td[td['year'] == year].shape[0]
    total_amount = td.shape[0]
    
    print(f"{year} makes up {(year_amount / total_amount) * 100}% of the dataset")

## Eliminate Off-Trend Data

It's clear that this dataset is HEAVILY weighted towards 2018. For that reason, dropping anything from before 2018 can help avoid skewing the data towards old trends, while keeping anything newer than 2018 might reveal new trends. 

If a dataset of such massive size consists of 99% of the same year, it's likely that the trips from newer years are either invalid data upon collection, and incomplete enough to actually show any trends.  

All rows but 2018 are, therefore, dropped. 

In [ ]:
td = td[td['year'] == 2018]
# Evaluate data stats after dropping
td.describe()

## Trip Fare - Sanity Checks

The value of *total_amount* should be equal to the sum of the *fare_amount*, *mta_tax*, *tip_amount*, *tolls_amount*, *imp_surcharge* and the *extra*.

Calculating total amounts and dropping rows whose values don't "add up"...

### Drop Fare Columns with Negative Values

In [ ]:
init_count = len(td)
td = td[td['fare_amount'] >= 0]
td = td[td['extra'] >= 0]
td = td[td['mta_tax'] >= 0]
td = td[td['tip_amount'] >= 0]
td = td[td['imp_surcharge'] >= 0]
td = td[td['tolls_amount'] >= 0]
final_count = len(td)

print(f"Fraction of dataframe retained: {final_count / init_count * 100}%")

### Verify that the Fare Values Add Up

In [ ]:
# Calculating total amounts and dropping rows whose values don't "add up"...
fare = td['fare_amount'].values
extra = np.add(fare, td['extra'].values)
mta_tax = np.add(extra, td['mta_tax'].values)
tip_amount = np.add(mta_tax, td['tip_amount'].values)
imp_surcharge = np.add(tip_amount, td['imp_surcharge'].values)
calculated_total_amount = np.add(imp_surcharge, td['tolls_amount'].values)

td['calculated_total_amount'] = pd.Series(calculated_total_amount)

# validate calculated total by manually adding all relevant columns and comparing to the calculated column
td.head(10)

Dropping incorrect `total_amount` values

In [ ]:
# Dropping incorrect `total_amount` values
init_count = len(td)
td = td[td['total_amount'] != td['calculated_total_amount']]
final_count = len(td)

print(f"Fraction of dataframe retained: {final_count / init_count * 100}%")

# Drop the computed fare value
td.drop('calculated_total_amount', axis=1, inplace=True)

td.describe()

## Dataframe After Cleanup

In [ ]:
# Display sample rows from the dataset
td.head()

# Data Transformation

The following data transformation steps are applied to the cleansed data, to facilitate further data analysis and mining.

## Construct Composite/Derived Attributes

Construction of composite and derived attributes can greatly aid the the learning and analysis phase of data mining. Simplex relationships across the data attributes that may not be captured by the downstream analysis models, can be expressed through explicity computed attributes through a combination of one or more pre-existing attributes.

#### Compute and Add Driving Speed

A `driving_speed` attribute in addition to the existing data attributes can be useful to analyze traffic data in different geographic locations of the city. Average speed can be directly correlated with the traffic density.

In [ ]:
trip_distance_array = td['trip_distance']
trip_duration_array = td['trip_duration']

# driving_speed = trip_dist (in miles) / trip_duration (in sec) * 3600 sec
driving_speed = np.divide(trip_distance_array, trip_duration_array)*3600

td['driving_speed'] = pd.Series(trip_duration)

#### Compute and Add Tipping Rate

`tipping_rate` can help to analyze the general proportion of tipping that cab riders usually pay for their rides.

In [ ]:
tip_amount_array = td['tip_amount']
total_amount_array = td['total_amount']

# tipping rate = tip_amount / total_amount
tipping_rate = np.divide(tip_amount_array, total_amount_array)

td['tipping_rate'] = tipping_rate

## Data Normalization

Data Normalization is a typical practice in data mining technique which consists of transforming numeric columns to a standard scale. Since some feature values differ from others multiple times, the features with higher values will dominate the learning and analysis process. Hence, bringing them down to the same scale is useful for faster and meaningful analysis.

Simple `min-max scaling` procedure is adopted in the following cells to normalize specific attributes. Since statistical information about the mean, variance, etc. is not known, more complex and informed scaling procedures cannot be applied.

### All Fare Attributes

All cost related columns have very distinct scales. `tip_amount`, for instance, is exteremely low while the `total_fare` is typically a very high value. Hence, the latter would dominate the analysis and model training (for ML) processes. These are normalized here,

In [ ]:
td_min_max_scaled = td.copy()
  
columns = [
    'fare_amount',
    'extra',
    'mta_tax',
    'tip_amount',
    'tolls_amount',
    'total_amount'
]
for column in columns:
  td_min_max_scaled[column] = (td_min_max_scaled[column] - td_min_max_scaled[column].min()) / (td_min_max_scaled[column].max() - td_min_max_scaled[column].min())    

td_min_max_scaled.head()

del td
td = td_min_max_scaled

## Numeric Encoding for Categorical String Attributes

Encoding categorical data is a process of converting categorical data into integer format so that the data with converted categorical values can be provided to the models to give and improve the predictions.

### Encode `store_and_fwd_flag` Attribute

In [1]:
td_encoded = td.copy()

td.store_and_fwd_flag = td.store_and_fwd_flag.astype('category').cat.codes

del td
td = td_encoded
td.head()

NameError: ignored

# Finishing Up  

The *total_amount* column did a lot more than just clean totals, but it actually checked all of the other total effecting columns at the same time. If any errors occurred in any column, the calculated total would have differed from the calculated total. 

Missing *mta_tax*, and incorrect *toll_amount* values are dropped. 

In [ ]:
# this is a quick, easy way to de-allocate the memory assigned to df, which holds the original dataframe
# this was necessary else the write to csv function of Pandas (to_csv) would max out the allowed memory in the notebook environment on Kaggle.
df=[]

Save the cleaned dataframe to a CSV.

In [ ]:
td.to_csv(os.path.join(DATA_PATH, 'taxi-trip-data_2018_cleaned.csv'))
print('Done!')

# Save the notebook

In [38]:
!apt-get install texlive texlive-xetex texlive-latex-extra pandoc
!pip install pypandoc

Reading package lists... Done
Building dependency tree       
Reading state information... Done
pandoc is already the newest version (1.19.2.4~dfsg-1build4).
texlive is already the newest version (2017.20180305-1).
texlive-latex-extra is already the newest version (2017.20180305-2).
texlive-xetex is already the newest version (2017.20180305-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [37]:
os.chdir("/content/drive/MyDrive/Undergrad/Semester-7/DWDM_Preprocessing-Assignment")
!ls
!jupyter nbconvert --to PDF "Analysis-Cleaning-Transformation.ipynb"

# Save processed CSV
td.to_csv(os.path.join(DATA_PATH, 'taxi-trip-data_2018_cleaned.csv'))
print('Done!')

1_Analysis.ipynb  Analysis-Cleaning-Transformation.ipynb
2_Cleaning.ipynb  Data
[NbConvertApp] Converting notebook Analysis-Cleaning-Transformation.ipynb to PDF
[NbConvertApp] Support files will be in Analysis-Cleaning-Transformation_files/
[NbConvertApp] Making directory ./Analysis-Cleaning-Transformation_files
[NbConvertApp] Writing 110780 bytes to ./notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', './notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', './notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 218153 bytes to Analysis-Cleaning-Transformation.pdf
Done!
